众所周知，如果你想复现一个 bug ， 重新运行 simulation 十分费时，特别是对于运行时间很长的负载，比如 SPEC CPU

As we know, if you want to reproduce a bug, re-run simulation is so time-consuming, especially for long workload like spec cpu.

So Snapshot is the way out. We have LightSSS, a light-weight simulation snapshot.

- During simulation, LightSSS will record snapshots of the process with funtion fork().
- When bug is detected, it will be waked up and 
- generate waveform of several cycles from the latest snapshot before the bug occurred.

因此我们需要快照， LightSSS 是一个轻量的仿真快照。

- 在仿真中， LightSSS 会通过 `fork()` 记录程序的快照。
- 当发现 bug 时，会唤醒并且
- 从 bug 发生前的最新快照中生成波形。

LightSSS具有良好的可扩展性，因为它允许对任何外部模型（例如用C++编写的模型）进行快照操作，且无需了解模型细节。此外，创建快照的开销极低，仅需约500微秒。这远低于Verilator的RTL快照开销。

LightSSS have good scalability because you can make snapshots for any external models (such as model written in C++), and do not need to understand model details.And the overhead of taking a snapshot is low, only about 500 micro-seconds. This is far less than the overhead of RTL snapshots from verilator.


![LightSSS](../images/02-functional/03-lightSSS/lightSSS-overall-en.png)

In [ ]:
%%bash
cd .. && source env.sh
cd ${NOOP_HOME}

mkdir -p build
rm -f ./build/*.vcd

$(get_asset emu-precompile/emu-alu-err) \
    -i $(get_asset workload/hello-riscv64-xs.bin) \
    --diff ${NEMU_HOME}/build/riscv64-nemu-interpreter-so \
    --enable-fork \
    2> /dev/null || true

echo -n "Dump wave: "
realpath ./build/*.vcd

如果您看到“the oldest checkpoint start to dump wave and dump nemu log...”，则表示 LightSSS 已生效。此后，模拟将从现有的最新快照重新开始并记录波形。

If you see "the oldest checkpoint start to dump wave and dump nemu log...", LightSSS is active. The simulation will then restart from the latest snapshot and record waveforms.